## Assignment 4: 

In [5]:
import numpy as np

from ase import Atoms
from ase.io.trajectory import Trajectory
from gpaw import GPAW, PW
from ase.io import read

### Task 1/2:

- Gold/Platinum/Rhodium - FCC lattice:

In [ ]:
a = 4.0   # Approximate lattice constant
b = a / 2 # FCC lattice

for Elem in ['Au', 'Pt', 'Rh']:
    calc = GPAW(xc   = 'PBE', 
            mode = PW(450),
            kpts = [12, 12, 12],
            txt  = f'T1_{Elem}.txt')
            
    Elem_Atoms = Atoms(f'{Elem}',
               cell = [(0, b, b), (b, 0, b), (b, b, 0)], # FCC lattice
               pbc  = 1, # Periodicity in all directions. I think that's what this means. Check latter.
               calculator = calc)
    cell = Elem_Atoms.get_cell()
    traj = Trajectory('T1_Au.traj', 'w')
#     print(Elem_Atoms.get_volume()) # Debugging.
#     print(Elem_Atoms.get_cell())   # Debugging.
    for x in np.linspace(0.80, 1.20, 40):
        Elem_Atoms.set_cell(cell * x, scale_atoms=True)
        Elem_Atoms.get_potential_energy()
        traj.write(Elem_Atoms)

In [2]:
# These are the correct values...
vols = [17.891, 15.594, 14.175]
[(4*vol)**(1/3) for vol in vols]

[4.151753263156935, 3.965876389260916, 3.8417374949624827]

In [12]:
# These are the correct values...
energies   = [-3.145, -6.453, -7.318]
Tot_En     = [-0.158, -0.704, -1.218]
Diff       = [-(energies[i] - Tot_En[i]) for i in range(3)]; formatted_Diff = [float(f'{item:.3f}') for item in Diff]
print(f'Cohesion energies = {formatted_Diff}.')

Cohesion energies = [2.987, 5.749, 6.1].


In [29]:
# Not these...
# Determining minima:
print('Task 2: Reading results from Task 1...')
# Total energies, from Task 2:
Tot_En     = [-0.158, -0.704, -1.218]
Elements   = ['Au', 'Pt', 'Rh']
Exp_Vals   = [3.81, 5.84, 5.75]       # Taken from: https://www.knowledgedoor.com/2/elements_handbook/cohesive_energy.html
Exp_Vals_a = [4.0782, 3.9242, 3.8034] # Taken from: https://periodictable.com/Properties/A/LatticeConstants.html 
for Tot_E, Elem, Exp, Exp_a in zip(Tot_En, Elements, Exp_Vals, Exp_Vals_a):
    configs   = read(f'T1_{Elem}.traj@0:40')  # Read all 40 configurations.
    volumes   = [Conf.get_volume() for Conf in configs]
    # energies  = [Conf.get_potential_energy() for Conf in configs]
    print(f'[{Elem}] Vol. = {volumes[np.argmin(energies)]:.5f} [\AA^3]; Lattice constant: a = {(4*volumes[np.argmin(energies)])**(1/3):.5f} [\AA] Exp. value_a = {Exp_a:.4f} [\AA].')
    print(f'[{Elem}] Energy = {min(energies):.5f} [eV]; Cohesive energy = {Tot_E:.5f} - ({min(energies):.5f}) = {Tot_E - min(energies):.2f} [eV]. Exp. value_E = {Exp:.2f} [eV].\n')

Task 2: Reading results from Task 1...
[Au] Vol. = 18.31921 [\AA^3]; Lattice constant: a = 4.18462 [\AA] Exp. value_a = 4.0782 [\AA].
[Au] Energy = -3.14578 [eV]; Cohesive energy = -0.15800 - (-3.14578) = 2.99 [eV]. Exp. value_E = 3.81 [eV].

[Pt] Vol. = 15.75511 [\AA^3]; Lattice constant: a = 3.97949 [\AA] Exp. value_a = 3.9242 [\AA].
[Pt] Energy = -6.43337 [eV]; Cohesive energy = -0.70400 - (-6.43337) = 5.73 [eV]. Exp. value_E = 5.84 [eV].

[Rh] Vol. = 14.33804 [\AA^3]; Lattice constant: a = 3.85641 [\AA] Exp. value_a = 3.8034 [\AA].
[Rh] Energy = -7.30502 [eV]; Cohesive energy = -1.21800 - (-7.30502) = 6.09 [eV]. Exp. value_E = 5.75 [eV].



In [17]:
# This one doesn't seem to be working...
from ase.build import bulk
from gpaw import GPAW, PW
import numpy as np

gold_atoms = bulk('Au', 'fcc', a = 4, cubic = True) # This function could be useful later on. Maybe...
calc = GPAW(xc   = 'PBE',                           # Cubic = True? I don't think this should be here!
            mode = PW(450),
            kpts = [12, 12, 12],
            txt  = 'gpaw.bulk.Au.txt')
gold_atoms.calc  = calc
gold_atoms.get_potential_energy()
calc.write('bulk.Au.gpw')
gold_atoms.cell # Check cell.

## Tsting for Task 3:

In [15]:
from ase.build import fcc111
import numpy as np

In [16]:
slab = fcc111(f'Au', size = (3, 3, 3), a = 4, vacuum = 6.0)
surface_area   = 2 * (slab.cell[0, 0] * slab.cell[1, 1])
print(surface_area)
slab.cell

124.70765814495917


Cell([[8.485281374238571, 0.0, 0.0], [4.242640687119286, 7.348469228349534, 0.0], [0.0, 0.0, 16.618802153517006]])

In [17]:
c = slab.cell[0, 0]/3
2*np.sqrt(2)*c

8.000000000000002

In [18]:
0.328420 * surface_area

40.95648908796749

Should the surface energy really be half the cohesive energy, under these conditions? Maybe ask teacher...